In [1]:
import traci
import os
import sys
import keyboard
import time

# --- Configuration ---
if 'SUMO_HOME' in os.environ:
    tools = os.path.join(os.environ['SUMO_HOME'], 'tools')
    sys.path.append(tools)
else:
    sys.exit("Please declare environment variable 'SUMO_HOME'")

sumoBinary = "sumo-gui"  # Utilisez 'sumo-gui' pour voir la simulation
sumoCmd = [sumoBinary, "-c", "ff_heterogeneous.sumocfg"]

In [4]:
def run_simulation():
    traci.start(sumoCmd)
    
    # 1. On définit la route une seule fois au début
    traci.route.add("bus_route_2", ["E0E1", "E1E2", "E2E3", "E3E4", "E4E5"])
    
    bus_count = 0
    
    while traci.simulation.getMinExpectedNumber() > 0:
        traci.simulationStep() # Avancer d'une seconde (ou d'un pas)
        
        sim_time = traci.simulation.getTime()
        
        # 2. Vérifier si on doit ajouter un bus (toutes les 180s à partir de 180s)
        if sim_time >= 180 and sim_time % 180 == 0:
            bus_id = f"bus_2.{bus_count}"
            traci.vehicle.add(bus_id, "bus_route_2", typeID="bus")
            traci.vehicle.setVehicleClass(bus_id, "bus")
            print(f"Bus {bus_id} ajouté à t={sim_time}")
            bus_count += 1
            
        # Optionnel : Sortie manuelle avec la touche 'q'
        if keyboard.is_pressed('q'):
            break
            
    traci.close()

In [5]:
# --- Lancement ---
try:
    run_simulation()
except traci.exceptions.FatalTraCIError:
    print("La simulation a été fermée.")

TraCIException: Connection 'default' is already active.